<a href="https://colab.research.google.com/github/moh2236945/pytorch_classification/blob/master/models/MobileNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import os
import shutil
from collections import OrderedDict
from IPython.display import clear_output

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader, random_split

In [3]:
class LambdaLayer(nn.Module):
    
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd
    
    def forward(self, x):
        return self.lambd(x)
    
class Bottleneck(nn.Module):
    
    def __init__(self, in_channels, out_channels, t, stride):
        super(Bottleneck, self).__init__()
        
        self.stride = stride
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        self.features = nn.Sequential(OrderedDict([
            ('pconv1', nn.Conv2d(in_channels,
                                in_channels*t,
                                kernel_size=1,
                                stride=1,
                                padding=0,
                                bias=False)),
            ('bn1', nn.BatchNorm2d(in_channels*t)),
            ('act1', nn.ReLU6()),
            ('dconv', nn.Conv2d(in_channels*t,
                                in_channels*t,
                                kernel_size=3,
                                groups=in_channels*t,
                                stride=stride,
                                padding=1,
                                bias=False)),
            ('bn2', nn.BatchNorm2d(in_channels*t)),
            ('act2', nn.ReLU6()),
            ('pconv3', nn.Conv2d(in_channels*t,
                                out_channels,
                                kernel_size=1,
                                stride=1,
                                padding=0,
                                bias=False)),
            ('bn3', nn.BatchNorm2d(out_channels))
        ]))
            
    def forward(self, x):
        out = self.features(x)
        if self.stride == 1 and self.in_channels == self.out_channels:
            out += x 
        return out

class MobileNet(nn.Module):
    
    def __init__(self, block_type, bottleneck_settings, width_multiplier, num_classes):
        super(MobileNet, self).__init__()
        
        self.num_classes = num_classes
        self.b_s = bottleneck_settings
        self.b_s['c'] = [int(elt * width_multiplier) for elt in self.b_s['c']]
        self.in_channels = int(32 * width_multiplier)
        self.out_channels = int(1280 * width_multiplier)
        
        # Feature
        self.conv0 = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(3, self.in_channels, 1, stride=2, bias=False)),
            ('bn0', nn.BatchNorm2d(self.in_channels)),
            ('act0', nn.ReLU6()) 
        ]))
        self.bottleneck1 = self.__build_layer(block_type,
                                              self.in_channels, 
                                              self.b_s['c'][0], 
                                              self.b_s['t'][0],
                                              self.b_s['s'][0],
                                              self.b_s['n'][0])
        self.bottleneck2 = self.__build_layer(block_type, 
                                              self.b_s['c'][0],
                                              self.b_s['c'][1], 
                                              self.b_s['t'][1],
                                              self.b_s['s'][1],
                                              self.b_s['n'][1])
        self.bottleneck3 = self.__build_layer(block_type,
                                              self.b_s['c'][1],
                                              self.b_s['c'][2],
                                              self.b_s['t'][2],
                                              self.b_s['s'][2],
                                              self.b_s['n'][2])
        self.bottleneck4 = self.__build_layer(block_type,
                                              self.b_s['c'][2],
                                              self.b_s['c'][3],
                                              self.b_s['t'][3],
                                              self.b_s['s'][3],
                                              self.b_s['n'][3])
        self.bottleneck5 = self.__build_layer(block_type,
                                              self.b_s['c'][3],
                                              self.b_s['c'][4],
                                              self.b_s['t'][4],
                                              self.b_s['s'][4],
                                              self.b_s['n'][4])
        self.bottleneck6 = self.__build_layer(block_type,
                                              self.b_s['c'][4],
                                              self.b_s['c'][5],
                                              self.b_s['t'][5],
                                              self.b_s['s'][5],
                                              self.b_s['n'][5])
        self.bottleneck7 = self.__build_layer(block_type,
                                              self.b_s['c'][5],
                                              self.b_s['c'][6],
                                              self.b_s['t'][6],
                                              self.b_s['s'][6],
                                              self.b_s['n'][6])
        # Classifier
        self.conv8 = nn.Sequential(OrderedDict([
            ('conv8', nn.Conv2d(self.b_s['c'][6], self.out_channels, 1, bias=False)),
            ('bn8', nn.BatchNorm2d(self.out_channels)),
            ('act8', nn.ReLU6()) 
        ]))
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.conv9 = nn.Conv2d(self.out_channels, num_classes, 1)
    
    def __build_layer(self, block_type, in_channels, out_channels, t, s, n):
        layers = []
        tmp_channels = in_channels
        for i in range(n):
            if i == 0:
                layers.append(block_type(tmp_channels, out_channels, t, s))
            else:
                layers.append(block_type(tmp_channels, out_channels, t, 1))
            tmp_channels = out_channels
        return nn.Sequential(*layers)
        
    def forward(self, x):
        out = self.conv0(x)
        out = self.bottleneck1(out)
        out = self.bottleneck2(out)
        out = self.bottleneck3(out)
        out = self.bottleneck4(out)
        out = self.bottleneck5(out)
        out = self.bottleneck6(out)
        out = self.bottleneck7(out)
        out = self.conv8(out)
        out = self.avgpool(out)
        out = self.conv9(out)
        out = out.view(-1, self.num_classes)
        return out

def MobileNetV2():
    bottleneck_settings = {
                'c': [16, 24, 32, 64, 96, 160, 320],
                't': [1, 6, 6, 6, 6, 6, 6],
                's': [1, 2, 2, 2, 1, 2, 1],
                'n': [1, 2, 3, 4, 3, 3, 1]
            }
    
    return MobileNet(block_type=Bottleneck,
                     bottleneck_settings=bottleneck_settings,
                     width_multiplier=.5,
                     num_classes=1000)



In [4]:
model = MobileNetV2()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
summary(model, (3, 96, 96))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 48, 48]              48
       BatchNorm2d-2           [-1, 16, 48, 48]              32
             ReLU6-3           [-1, 16, 48, 48]               0
            Conv2d-4           [-1, 16, 48, 48]             256
       BatchNorm2d-5           [-1, 16, 48, 48]              32
             ReLU6-6           [-1, 16, 48, 48]               0
            Conv2d-7           [-1, 16, 48, 48]             144
       BatchNorm2d-8           [-1, 16, 48, 48]              32
             ReLU6-9           [-1, 16, 48, 48]               0
           Conv2d-10            [-1, 8, 48, 48]             128
      BatchNorm2d-11            [-1, 8, 48, 48]              16
       Bottleneck-12            [-1, 8, 48, 48]               0
           Conv2d-13           [-1, 48, 48, 48]             384
      BatchNorm2d-14           [-1, 48,